# SPX volatility

The aim of this notebook is to investigate the volatility of SPX and to see whether the measured volatility matches the implied volatilities observed through options. The following steps will be undertaken:

- **Long history**: I compare different moving average windows in order to see what can be said about the recommended window of 90 or 180 days
- **Day count**: I investigate whether non-trading days have lower volatility than trading days and whether changes on the last trading day before a non trading day have a different volatility than other trading days (also based on a comment in the book)
- **Implied volatilities**: A simple comparison between implied and (backward-)measured volatilities.
- **Volatility smiles**: A comparison of implied volatilities for different strikes to compare against the expectation from the book.

In [42]:
from datetime import datetime
from io import StringIO
import os

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import iplot
import requests


## History

In [58]:
csv = requests.get("https://raw.githubusercontent.com/o1i/hull/main/data/2012-12-13_spx_historic.csv").content.decode("utf-8")


In [9]:
spx_hist = pd.read_csv(StringIO(csv))
dt_fmt = "%Y-%m-%d"
spx_hist["date_dt"] = spx_hist["date"].map(lambda x: datetime.strptime(x, dt_fmt))
spx_hist.sort_values("date_dt", inplace=True)
spx_hist.set_index("date_dt", inplace=True)

# The first 15 years or so open = close --> to be excluded
first_close_unlike_open = list(~(spx_hist["open"] == spx_hist["close"])).index(True)
spx_hist_short = spx_hist[first_close_unlike_open:]

### Intra Day moves

In [14]:
intra_day = np.log10(spx_hist_short["close"] / spx_hist_short["open"])

In [16]:
trace_22d = go.Scatter(
    x=spx_hist_short.index,
    y=intra_day.rolling(22).std(),
    mode="lines"
)
layout = {
    'showlegend': True
}

fig = {
    'layout': layout,
    'data': [trace_22d],
}

iplot(fig)

In [15]:
len(intra_day)

3258

In [32]:
len(spx_hist.index)

7621

In [35]:
spx_hist.index

RangeIndex(start=0, stop=7621, step=1)

In [48]:
data = list(range(10))
trace = go.Scatter(
    x=list(range(len(data))),
    y=data,
    mode="lines"
)
layout = {
    'shapes': [
        # Line Horizontal
        {
            'type': 'line',
            'x0': 0,
            'y0': 4,
            'x1': 10,
            'y1': 4,
            'line': {
                'color': 'rgb(50, 171, 96)',
                'width': 4
            },
        }
    ],
    'showlegend': True
}

fig = {
    'layout': layout,
    'data': [trace],
}


iplot(fig)